In [1]:
import numpy as np
import pandas as pd
from sympy import diff, lambdify, symbols, log, sqrt
from sympy.stats import Normal, density
from scipy.stats import norm

kappa = 0.1465
alpha = 0.5172
omega = 0.5786
rho = -0.0243
s0 = 1000
r = 0
T = 1 / 12
K = 1000
xi = 0.6
# symbolic variables
s, v, t, v0 = symbols('s v t v0')
N = Normal('x', 0, 1)
d_p = (log(s / K) + (r + v0 / 2) * t) / sqrt(v0 * t)
D0 = 0.5 * (v - v0) * s * density(N)(d_p) / sqrt(v0 * t)
delta0 = diff(D0, s)
gamma0 = diff(delta0, s)
vega0 = diff(D0, v)

def delta(S, K, T, r, sigma):
  N = norm.cdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  return N(d_p)

def vega(S, K, T, r, sigma):
  N = norm.pdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  return S * N(d_p) * np.sqrt(T) / (2 * sigma)

def L(f): # infinitesimal generator
  ft = diff(f, t)
  fs = diff(f, s)
  fs2 = diff(fs, s)
  fv = diff(f, v)
  fv2 = diff(fv, v)
  fvs = diff(fv, s)
  tmp1 = -ft + r * s * fs + kappa * (alpha - v) * fv
  tmp2 = (v * s**2 * fs2 + omega**2 * v**(2 * xi) * fv2) / 2
  tmp3 = rho * omega * v**(xi + 0.5) * s * fvs
  return tmp1 + tmp2 + tmp3 - r * f

D = [D0]
for _ in range(1, 5):
  D.append(L(D[-1]))

In [2]:
Delta = [delta0]
Delta_func = []
for i in range(1, 5):
  Delta.append(diff(D[i], s))
for i in range(0, 5):
  Delta_func.append(lambdify([s, v, t, v0], Delta[i]))
  
Vega = [vega0]
Vega_func = []
for i in range(1, 5):
  Vega.append(diff(D[i], v))
for i in range(0, 5):
  Vega_func.append(lambdify([s, v, t, v0], Vega[i]))

In [3]:
%%time
V = np.arange(0.1, 1.2, 0.1)
Approx_delta = []
for v in V:
  delta0_approx = delta(s0, K, T, r, np.sqrt(v)) + T * Delta_func[0](s0, v, T, v)
  delta1_approx = delta0_approx + T**2 / 2 * Delta_func[1](s0, v, T, v)
  delta2_approx = delta1_approx + T**3 / 6 * Delta_func[2](s0, v, T, v)
  delta3_approx = delta2_approx + T**4 / 24 * Delta_func[3](s0, v, T, v)
  delta4_approx = delta3_approx + T**5 / 120 * Delta_func[4](s0, v, T, v)
  Approx_delta.append(100*delta4_approx)
print(Approx_delta)

[51.9312951252368, 52.65175620082644, 53.2131019468454, 53.689075272389864, 54.10956835080122, 54.49024896272184, 54.84054813562813, 55.16666630142682, 55.47293886868868, 55.76253858110333, 56.03787042689967]
Wall time: 7.56 s


In [4]:
%%time
Approx_vega = []
for v in V:
  vega0_approx = vega(s0, K, T, r, np.sqrt(v))
  vega1_approx = vega0_approx + T**2 / 2 * Vega_func[1](s0, v, T, v)
  vega2_approx = vega1_approx + T**3 / 6 * Vega_func[2](s0, v, T, v)
  vega3_approx = vega2_approx + T**4 / 24 * Vega_func[3](s0, v, T, v)
  vega4_approx = vega3_approx + T**5 / 120 * Vega_func[4](s0, v, T, v)
  Approx_vega.append(vega4_approx)
print(Approx_vega)

[179.72178919212985, 127.47630573428944, 104.1366343343541, 90.16338118024586, 80.60006351321138, 73.52507974626425, 68.01625141792772, 63.56851232992541, 59.878957307965344, 56.75342764216735, 54.06096151299987]
Wall time: 6.2 s


In [5]:
Delta_MC = np.array([51.9501, 52.5276, 52.9595, 53.3255, 53.8027, 54.0987, 54.4164, 54.7429, 54.9393, 55.2099, 55.4269])
Delta_diff = (Approx_delta - Delta_MC) / Delta_MC * 100
Delta_JFE = np.array([51.9568, 52.6729, 53.2317, 53.7060, 54.1252, 54.5049, 54.8543, 55.1798, 55.4854, 55.7745, 56.0494])
Delta_JFE_diff = np.array([0.0131, 0.2767, 0.5140, 0.7135, 0.5995, 0.7508, 0.8047, 0.7981, 0.9941, 1.0227, 1.1230])

Vega_MC = np.array([177.4286, 126.1482, 103.1555, 89.3784, 79.9090, 72.9385, 67.4934, 63.0958, 59.4809, 56.3929, 53.7493])
Vega_diff = (Approx_vega - Vega_MC) / Vega_MC * 100
Vega_JFE = np.array([180.7478, 127.7828, 104.2902, 90.2579, 80.6650, 73.5728, 68.0529, 63.5975, 59.9025, 56.7729, 54.0773])
Vega_JFE_diff = np.array([1.8707, 1.2958, 1.1000, 0.9841, 0.9460, 0.8696, 0.8290, 0.7952, 0.7089, 0.6739, 0.6103])

df = pd.DataFrame({
    "V": V, 
    "MC %delta": Delta_MC, 
    "Replicate %delta": np.round(Approx_delta, 4), 
    "Replicate delta %diff": np.round(Delta_diff, 4),
    "JFE %delta": Delta_JFE,
    "JFE delta %diff": Delta_JFE_diff,
    "MC %vega": Vega_MC,
    "Replicate %vega": np.round(Approx_vega, 4),
    "Replicate vega %diff": np.round(Vega_diff, 4),
    "JFE %vega": Vega_JFE,
    "JFE vega %diff": Vega_JFE_diff
    })
print(df)
df.to_csv("Panel AB2 (s=1000).csv")

      V  MC %delta  Replicate %delta  Replicate delta %diff  JFE %delta  \
0   0.1    51.9501           51.9313                -0.0362     51.9568   
1   0.2    52.5276           52.6518                 0.2364     52.6729   
2   0.3    52.9595           53.2131                 0.4789     53.2317   
3   0.4    53.3255           53.6891                 0.6818     53.7060   
4   0.5    53.8027           54.1096                 0.5704     54.1252   
5   0.6    54.0987           54.4902                 0.7238     54.5049   
6   0.7    54.4164           54.8405                 0.7794     54.8543   
7   0.8    54.7429           55.1667                 0.7741     55.1798   
8   0.9    54.9393           55.4729                 0.9713     55.4854   
9   1.0    55.2099           55.7625                 1.0010     55.7745   
10  1.1    55.4269           56.0379                 1.1023     56.0494   

    JFE delta %diff  MC %vega  Replicate %vega  Replicate vega %diff  \
0            0.0131  177.42